In [1]:
## create our model

In [2]:
import pandas as pd

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, Conv1D, MaxPool1D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

from collections import Counter
from sklearn.cluster import KMeans
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
photo_info = pd.read_csv('Behance\\info_color_photos_simple.csv')

In [4]:
photo_info.head()

,Unnamed: 0,Unnamed: 0.1,path,image_nr,likes,visualizations,date,simp_path,day,year,month,month_n,likes_n,visualizations_n,colors,colors_s
0,0,0,Behance\0_1.8k_6.4k_84_3 de agosto de 2021.jpg,0,1.8k,6.4k,3 de agosto de 2021,0_1.8k_6.4k_84_3 de agosto de 2021.jpg,3,2021,agosto,8,1800,6400,[[213.44514 181.73178 200.50743 ]\n [150.139...,[[0.5890737 0.66480434 0.8062853 ]\n [0.76469...
1,1,1,Behance\1000_765_8.3k_65_19 de abril de 2021.jpg,1000,765,8.3k,19 de abril de 2021,1000_765_8.3k_65_19 de abril de 2021.jpg,19,2021,abril,4,765,8300,[[ 59.659153 39.769173 35.604183]\n [115.539...,[[0.450158 0.47198504 0.55983007]\n [0.23607...
2,2,2,Behance\1001_919_6k_63_19 de abril de 2021.jpg,1001,919,6k,19 de abril de 2021,1001_919_6k_63_19 de abril de 2021.jpg,19,2021,abril,4,919,6000,[[240.91878 66.80268 51.22841 ]\n [ 31....,[[0.4243812 0.4627723 0.35667711]\n [0.97356...
3,3,3,Behance\1002_2.9k_21.9k_135_19 de abril de 202...,1002,2.9k,21.9k,19 de abril de 2021,1002_2.9k_21.9k_135_19 de abril de 2021.jpg,19,2021,abril,4,2900,21900,[[197.10681 237.85242 78.1398 ]\n [ 16.734...,[[0.06562523 0.16324136 0.03473371]\n [0.59959...
4,4,4,Behance\1003_928_9.9k_80_8 de abril de 2021.jpg,1003,928,9.9k,8 de abril de 2021,1003_928_9.9k_80_8 de abril de 2021.jpg,8,2021,abril,4,928,9900,[[211.45775 207.38548 206.43594 ]\n [ 68.940...,[[0.9506851 0.95043564 0.9497987 ]\n [0.44098...


In [5]:
photo_info = photo_info.drop(['Unnamed: 0'], axis=1)
photo_info = photo_info.drop(['Unnamed: 0.1'], axis=1)
photo_info = photo_info.drop(['image_nr'], axis=1)
photo_info = photo_info.drop(['likes'], axis=1)
photo_info = photo_info.drop(['visualizations'], axis=1)
photo_info = photo_info.drop(['date'], axis=1)
photo_info = photo_info.drop(['month'], axis=1)
photo_info = photo_info.drop(['colors'], axis=1)
photo_info = photo_info.drop(['colors_s'], axis=1)

In [6]:
photo_info['comp_path'] = ['Behance//'+str(image) for image in photo_info['simp_path']]  ## get the colors of the image
photo_info.head()

,path,simp_path,day,year,month_n,likes_n,visualizations_n,comp_path
0,Behance\0_1.8k_6.4k_84_3 de agosto de 2021.jpg,0_1.8k_6.4k_84_3 de agosto de 2021.jpg,3,2021,8,1800,6400,Behance//0_1.8k_6.4k_84_3 de agosto de 2021.jpg
1,Behance\1000_765_8.3k_65_19 de abril de 2021.jpg,1000_765_8.3k_65_19 de abril de 2021.jpg,19,2021,4,765,8300,Behance//1000_765_8.3k_65_19 de abril de 2021.jpg
2,Behance\1001_919_6k_63_19 de abril de 2021.jpg,1001_919_6k_63_19 de abril de 2021.jpg,19,2021,4,919,6000,Behance//1001_919_6k_63_19 de abril de 2021.jpg
3,Behance\1002_2.9k_21.9k_135_19 de abril de 202...,1002_2.9k_21.9k_135_19 de abril de 2021.jpg,19,2021,4,2900,21900,Behance//1002_2.9k_21.9k_135_19 de abril de 20...
4,Behance\1003_928_9.9k_80_8 de abril de 2021.jpg,1003_928_9.9k_80_8 de abril de 2021.jpg,8,2021,4,928,9900,Behance//1003_928_9.9k_80_8 de abril de 2021.jpg


In [7]:
def color_extractor_simple(path):
    try: 
        image = cv2.imread(path,1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # reshape image
        pixels = image.reshape((-1,3)).astype("float32") / 255
        # Cluster the pixels
        centers = KMeans(n_clusters=10).fit(pixels).cluster_centers_
        res = (centers) #.astype("float64")
    except:
        res = 'Not_found'
    return res

In [8]:
photo_info['colors_s'] = [color_extractor_simple(image) for image in photo_info['comp_path']]  ## get the colors of the image
photo_info.head()

,path,simp_path,day,year,month_n,likes_n,visualizations_n,comp_path,colors_s
0,Behance\0_1.8k_6.4k_84_3 de agosto de 2021.jpg,0_1.8k_6.4k_84_3 de agosto de 2021.jpg,3,2021,8,1800,6400,Behance//0_1.8k_6.4k_84_3 de agosto de 2021.jpg,"[[0.40873614, 0.44456628, 0.5715072], [0.61313..."
1,Behance\1000_765_8.3k_65_19 de abril de 2021.jpg,1000_765_8.3k_65_19 de abril de 2021.jpg,19,2021,4,765,8300,Behance//1000_765_8.3k_65_19 de abril de 2021.jpg,"[[0.45240125, 0.47469607, 0.5625806], [0.11052..."
2,Behance\1001_919_6k_63_19 de abril de 2021.jpg,1001_919_6k_63_19 de abril de 2021.jpg,19,2021,4,919,6000,Behance//1001_919_6k_63_19 de abril de 2021.jpg,"[[0.61931163, 0.6110128, 0.5122256], [0.211172..."
3,Behance\1002_2.9k_21.9k_135_19 de abril de 202...,1002_2.9k_21.9k_135_19 de abril de 2021.jpg,19,2021,4,2900,21900,Behance//1002_2.9k_21.9k_135_19 de abril de 20...,"[[0.708725, 0.37450188, 0.9714448], [0.0656263..."
4,Behance\1003_928_9.9k_80_8 de abril de 2021.jpg,1003_928_9.9k_80_8 de abril de 2021.jpg,8,2021,4,928,9900,Behance//1003_928_9.9k_80_8 de abril de 2021.jpg,"[[0.95066166, 0.9504024, 0.9497682], [0.124519..."


In [9]:
photo_info = photo_info[photo_info['colors_s'] != 'Not_found']

C:\Users\catar\anaconda3\envs\Ironhack\lib\site-packages\pandas\core\ops\array_ops.py:55: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [10]:
#photo_info.head()

In [11]:
photo_info['data'] = [[photo_info['colors_s'].iloc[i], photo_info['likes_n'].iloc[i]] for i in range(len(photo_info['likes_n']))]
photo_info.head()

,path,simp_path,day,year,month_n,likes_n,visualizations_n,comp_path,colors_s,data
0,Behance\0_1.8k_6.4k_84_3 de agosto de 2021.jpg,0_1.8k_6.4k_84_3 de agosto de 2021.jpg,3,2021,8,1800,6400,Behance//0_1.8k_6.4k_84_3 de agosto de 2021.jpg,"[[0.40873614, 0.44456628, 0.5715072], [0.61313...","[[[0.40873614, 0.44456628, 0.5715072], [0.6131..."
1,Behance\1000_765_8.3k_65_19 de abril de 2021.jpg,1000_765_8.3k_65_19 de abril de 2021.jpg,19,2021,4,765,8300,Behance//1000_765_8.3k_65_19 de abril de 2021.jpg,"[[0.45240125, 0.47469607, 0.5625806], [0.11052...","[[[0.45240125, 0.47469607, 0.5625806], [0.1105..."
2,Behance\1001_919_6k_63_19 de abril de 2021.jpg,1001_919_6k_63_19 de abril de 2021.jpg,19,2021,4,919,6000,Behance//1001_919_6k_63_19 de abril de 2021.jpg,"[[0.61931163, 0.6110128, 0.5122256], [0.211172...","[[[0.61931163, 0.6110128, 0.5122256], [0.21117..."
3,Behance\1002_2.9k_21.9k_135_19 de abril de 202...,1002_2.9k_21.9k_135_19 de abril de 2021.jpg,19,2021,4,2900,21900,Behance//1002_2.9k_21.9k_135_19 de abril de 20...,"[[0.708725, 0.37450188, 0.9714448], [0.0656263...","[[[0.708725, 0.37450188, 0.9714448], [0.065626..."
4,Behance\1003_928_9.9k_80_8 de abril de 2021.jpg,1003_928_9.9k_80_8 de abril de 2021.jpg,8,2021,4,928,9900,Behance//1003_928_9.9k_80_8 de abril de 2021.jpg,"[[0.95066166, 0.9504024, 0.9497682], [0.124519...","[[[0.95066166, 0.9504024, 0.9497682], [0.12451..."


In [12]:
## lets divide data in train and test.

#create train and test sets
#train_photos = photo_info['colors_s'][:-130]      ## 130
#test_photos = photo_info['colors_s'][-130:]      ## 130

#train_likes =  photo_info['likes_n'][:-130]    ## 1771
#test_likes = photo_info['likes_n'][-130:]      ## 130

#train_df = pd.DataFrame({'name': train_photos, 'values': train_likes})
#test_df = pd.DataFrame({'name': test_photos, 'values': test_likes})

In [13]:
# size we want our images 
#img_size = 224


#def get_data(df):
#    data = []
    
    #path = data_dir
#    itens = df['name']                 #[f for f in os.listdir(path) if not f.startswith('.')]         
    
#    for img in itens:

#        np.equal(train_df['name'],img).all(1)
        
#        value = df[df['name'][:] == img]['values']  #.values[0]
        
#        try:
            #img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
            #resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
#            data.append([img, value])
#        except Exception as e:
#            print(e)
#    return np.array(data)

In [14]:
#create train and test sets
train = photo_info['data'][:-130].to_numpy()      ## 130
test = photo_info['data'][-130:].to_numpy()     ## 130

In [15]:
#train = get_data(train_df)
#test = get_data(test_df)

In [16]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_val.append(feature)
    y_val.append(label)

# Normalize the data
x_train = np.array(x_train) #/ 255
x_val = np.array(x_val) #/ 255

# coverter as imagens num array unico
x_train.reshape(-1, 10, 3, 1)  ## verificar se é necessario
y_train = np.array(y_train)

x_val.reshape(-1, 10, 3, 1)
y_val = np.array(y_val)

In [17]:
x_train.shape

(992, 10, 3)

In [18]:
## Create Model

model1 = Sequential()
model1.add(Conv1D(10, 2,padding="same", activation="relu", input_shape=(10,3)))    ## mudar para 3,10
model1.add(MaxPool1D(padding='same'))

model1.add(Conv1D(10, 2, padding="same", activation="relu"))
model1.add(MaxPool1D(padding='same'))

model1.add(Conv1D(64, 2, padding="same", activation="relu"))
model1.add(MaxPool1D(padding='same'))
model1.add(Dropout(0.4))

model1.add(Flatten())
model1.add(Dense(128,activation="relu"))
#model.add(Dense(2, activation="softmax")) # for classification
model1.add(Dense(1, activation="relu"))# for regression

### --- other network --- ###

# model = Sequential()
# model.add(Dense(22,activation='relu', input_shape=(225,225,3)))
# model.add(Dense(33,activation='relu'))
# model.add(Dense(128,activation='relu'))
# model.add(Dense(55,activation='relu'))
# model.add(Flatten())


### -- not used -- ####
model1.compile(optimizer='Adam',loss='mean_squared_error')

In [19]:
## TRAIN THE MODEL ##

history = model1.fit(x=x_train,y=y_train,validation_data=(x_val,y_val),epochs=35)

Epoch 1/35
31/31 [==============================] - 1s 6ms/step - loss: 2582135.5000 - val_loss: 2319964.2500
Epoch 2/35
31/31 [==============================] - 0s 2ms/step - loss: 2496410.7500 - val_loss: 2098780.2500
Epoch 3/35
31/31 [==============================] - 0s 2ms/step - loss: 1857499.0000 - val_loss: 932360.3750
Epoch 4/35
31/31 [==============================] - 0s 2ms/step - loss: 769043.8125 - val_loss: 617048.2500
Epoch 5/35
31/31 [==============================] - 0s 2ms/step - loss: 695003.8125 - val_loss: 599117.3125
Epoch 6/35
31/31 [==============================] - 0s 2ms/step - loss: 696694.9375 - val_loss: 603735.8125
Epoch 7/35
31/31 [==============================] - 0s 2ms/step - loss: 696540.1875 - val_loss: 597906.4375
Epoch 8/35
31/31 [==============================] - 0s 2ms/step - loss: 697876.3125 - val_loss: 598522.5000
Epoch 9/35
31/31 [==============================] - 0s 2ms/step - loss: 692290.1875 - val_loss: 601137.6250
Epoch 10/35
31/31 [====

In [20]:
## EVALUTE MODEL
np.sqrt(model1.evaluate(x_val,y_val))

5/5 [==============================] - 0s 1ms/step - loss: 583995.5000


764.1959827164757

In [21]:
model1.save('model3')

INFO:tensorflow:Assets written to: model3\assets


In [22]:
## função para prever novos (preços/ stock) de imagens novas

#img_size = 224

#def predicting(path):   
#    all_new = []
#    img = [f for f in os.listdir(path) if not f.startswith('.')]
#    for im in img:
#        img_arr = cv2.imread(os.path.join(path,im))[...,::-1]
#        resized_arr = cv2.resize(img_arr, (img_size, img_size))
#        all_new.append(resized_arr)
#    norm = np.array(all_new)/255    
#    pred = model.predict(norm).reshape(1,-1)[0]    
#    return pred

#path = '../input/new_im'
#predicting(path)